The core idea of agents is to use a language model to choose a sequence of actions to take. In chains, a sequence of actions is hardcoded. In agents, a language model is used as a reasoning engine to determine which actions to take and in which order

# Quickstart
To best understand the agent framework, let's build an agent that has two tools: one to look things online, and one to look up specific data that we've loaded into an index.

## Setup: Langsmith
By definition, agents take a self-determined, input-dependent sequence of steps before returning  a user-facing output. This makes debugging these systems rather tricky, and observability particularly important. Langsmith is especially useful in such cases

In [1]:
# Load environment variables from .env file
import os
from dotenv import load_dotenv
load_dotenv()

# Access the value of LANGCHAIN_API_KEY from .env file
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

In [4]:
from langchain_community.tools.tavily_search import TavilySearchResults
search = TavilySearchResults()
search.invoke("Who was the leader of the Mau Mau Revolution in Kenya?")

[{'url': 'https://learnaboutkenya.com/who-were-the-leaders-of-the-mau-mau-movement/',
  'content': 'Jomo Kenyatta, born on October 20, 1897, in Gatundu, British East Africa, was a prominent leader during the Mau Mau movement in Kenya. Growing up in a traditional Kikuyu family, Kenyatta experienced firsthand the injustices and discrimination against indigenous Africans under British colonial rule. This early exposure to the hardships faced by ...'},
 {'url': 'https://en.wikipedia.org/wiki/Dedan_Kimathi',
  'content': 'Dedan Kimathi Waciuri (born Kimathi wa Waciuri; 31 October 1920 - 18 February 1957) was the senior military and spiritual leader of the Kikuyu rebels involved in the Mau Mau Uprising.Widely regarded as a revolutionary leader, he led the armed military struggle against the British colonial regime in Kenya in the 1950s until his capture in 1956 and execution in 1957.'},
 {'url': 'https://en.wikipedia.org/wiki/Mau_Mau_rebellion',
  'content': 't. e. The Mau Mau rebellion (195

## Retriever


In [7]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://en.wikipedia.org/wiki/Mau_Mau_rebellion")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
).split_documents(docs)
vector = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vector.as_retriever()

In [8]:
retriever.get_relevant_documents("Who was the leader of the Mau Mau rebellion")[0]

Document(page_content="Kenya portalvte\nThe Mau Mau rebellion (1952–1960), also known as the Mau Mau uprising, Mau Mau revolt, or Kenya Emergency, was a war in the British Kenya Colony (1920–1963) between the Kenya Land and Freedom Army (KLFA), also known as the Mau Mau, and the British authorities.[7] Dominated by Kikuyu, Meru and Embu fighters, the KLFA also comprised units of Kamba[8] and Maasai who fought against the European colonists in Kenya, the British Army, and the local Kenya Regiment (British colonists, local auxiliary militia, and pro-British Kikuyu).[9][b]\nThe capture of rebel leader Field Marshal Dedan Kimathi on 21 October 1956 signalled the defeat of the Mau Mau, and essentially ended the British military campaign.[10] However, the rebellion survived until after Kenya's independence from Britain, driven mainly by the Meru units led by Field Marshal Musa Mwariama. General Baimungi, one of the last Mau Mau leaders, was killed shortly after Kenya attained self-rule.[11]"

Now that we have populated our index that we will be doing retrieval over, we can easily turn it into a tool(the format needed for an agent to properly use it)

In [20]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "maumau_history",
    "Search for any information on the mau mau. You may use this tool"
)

# create a list of tools we will use downstream
tools = [search, retriever_tool]

# create the agent
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [21]:
from langchain import hub
# choose the prompt we will use to guide the agent
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

Now we can initialize the agent with the LLM, the prompt, and the tools. The agent is responsible for taking an input and deciding what actions to take. Crucially, the agent does not execute those actions- that is done by the AgentExecutor

In [22]:
from langchain.agents import create_openai_functions_agent

agent = create_openai_functions_agent(llm, tools, prompt)

We n ow combine the agent(brains) with the tools inside the AgentExecutor(which will repeatedly call the agent anmd execute the tools.)

In [23]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools = tools, verbose=True)

# run the agent
agent_executor.invoke({"input": "hi"})



> Entering new AgentExecutor chain...


Hello! How can I assist you today?

> Finished chain.


{'input': 'hi', 'output': 'Hello! How can I assist you today?'}

In [24]:
agent_executor.invoke({"input": "Who was the mau mau leader?"})



> Entering new AgentExecutor chain...

Invoking: `maumau_history` with `{'query': 'mau mau leader'}`


Mau Mau were the militant wing of a growing clamour for political representation and freedom in Kenya. The first attempt to form a countrywide political party began on 1 October 1944.[71] This fledgling organisation was called the Kenya African Study Union. Harry Thuku was the first chairman, but he soon resigned. There is dispute over Thuku's reason for leaving KASU: Bethwell Ogot says Thuku "found the responsibility too heavy";[71] David Anderson states that "he walked out in disgust" as the militant section of KASU took the initiative.[72] KASU changed its name to the Kenya African Union (KAU) in 1946. Author Wangari Maathai writes that many of the organizers were ex-soldiers who fought for the British in Ceylon, Somalia, and Burma during the Second World War. When they returned to Kenya, they were never paid and did not receive recognition for their service, whereas their Britis

{'input': 'Who was the mau mau leader?',
 'output': 'The Mau Mau movement did not have a single leader, but rather had various leaders and figures who played significant roles in the movement. Some of the prominent leaders associated with the Mau Mau movement include Field Marshal Mwariama and Field Marshal Baimungi. These leaders, along with other Mau Mau fighters, were involved in the resistance against colonial authorities in Kenya.'}

In [25]:
agent_executor.invoke({"input": "Is it correct to call Dedan Kimathi the leader of the Mau Mau Rebellion??"})



> Entering new AgentExecutor chain...

Invoking: `maumau_history` with `{'query': 'Dedan Kimathi'}`


Kanogo, Tabitha (1992). Dedan Kimathi: A Biography. Nairobi: East African Educational Publishers.
— (1993) [1987]. Squatters and the Roots of Mau Mau, 1905–63. Nairobi: East African Educational Publishers. ISBN 978-9966-46-326-5.
Kariuki, Josiah Mwangi (1975). "Mau Mau" Detainee: The Account by a Kenya African of his Experiences in Detention Camps 1953–1960. New York and London: Oxford University Press.
Leys, Norman (1973). Kenya (4th ed.). London: Frank Cass. ISBN 978-0-714-61688-9.
Lonsdale, John (2000). "KAU's Cultures: Imaginations of Community and Constructions of Leadership in Kenya after the Second World War". Journal of African Cultural Studies. 13 (1): 107–124. doi:10.1080/713674307. JSTOR 1771859. S2CID 143712816.
— (2003). "Authority, Gender & Violence: The war within Mau Mau's fight for land & freedom". In Elisha Stephen Atieno-Odhiambo & John Lonsdale (eds.). Mau Mau and

{'input': 'Is it correct to call Dedan Kimathi the leader of the Mau Mau Rebellion??',
 'output': 'Dedan Kimathi was a prominent figure in the Mau Mau Rebellion, but he was not the sole leader of the movement. The Mau Mau Rebellion was a complex and decentralized uprising involving multiple leaders and participants. Dedan Kimathi is often remembered as one of the key leaders of the Mau Mau movement, but it would be more accurate to describe him as a significant figure rather than the sole leader.'}

In [27]:
agent_executor.invoke({"input": "Describe the mau mau tactics in the fight for independence"})



> Entering new AgentExecutor chain...

Invoking: `maumau_history` with `{'query': 'mau mau tactics in the fight for independence'}`


Mau Mau warfare[edit]

Mau Mau were the militant wing of a growing clamour for political representation and freedom in Kenya. The first attempt to form a countrywide political party began on 1 October 1944.[71] This fledgling organisation was called the Kenya African Study Union. Harry Thuku was the first chairman, but he soon resigned. There is dispute over Thuku's reason for leaving KASU: Bethwell Ogot says Thuku "found the responsibility too heavy";[71] David Anderson states that "he walked out in disgust" as the militant section of KASU took the initiative.[72] KASU changed its name to the Kenya African Union (KAU) in 1946. Author Wangari Maathai writes that many of the organizers were ex-soldiers who fought for the British in Ceylon, Somalia, and Burma during the Second World War. When they returned to Kenya, they were never paid and did not recei

{'input': 'Describe the mau mau tactics in the fight for independence',
 'output': "The Mau Mau were the militant wing of a movement seeking political representation and freedom in Kenya. They used secretive violence as a tactic in their fight for independence. There is ongoing debate about the impact of the Mau Mau rebellion on decolonization and Kenya after independence. Some argue that Kenya's independence was a result of the British government's decision to avoid further conflict, while others believe that nationalist intransigence played a significant role in shaping British policy towards Kenyan nationalism."}

## Adding in memory
Currently, the agent is stateless. This means that it does not remember previous interactions. To give it a memory, we need to pass in previous chat_history

In [28]:
# here we will pass an empty list of messages for chat_history because it is the first message in the chat
agent_executor.invoke({"input": "Hi, my name is Kevin", "chat_history": []})



> Entering new AgentExecutor chain...
Hello Kevin! How can I assist you today?

> Finished chain.


{'input': 'Hi, my name is Kevin',
 'chat_history': [],
 'output': 'Hello Kevin! How can I assist you today?'}

In [29]:
from langchain_core.messages import AIMessage, HumanMessage
agent_executor.invoke(
    {
        "chat_history": [
            HumanMessage(content="Hi, my name is Kevin"),
            AIMessage(content = 'Hello Kevin! How can I assist you today?')
        ],
        "input": "What is my name"
    }
)



> Entering new AgentExecutor chain...
Your name is Kevin.

> Finished chain.


{'chat_history': [HumanMessage(content='Hi, my name is Kevin'),
  AIMessage(content='Hello Kevin! How can I assist you today?')],
 'input': 'What is my name',
 'output': 'Your name is Kevin.'}

If we want to keep track of these messages automatically, we can wrap this in a RunnableWithMessageHistory

In [31]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

message_history = ChatMessageHistory()

agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    lambda session_id: message_history,
    input_message_key = "input",
    history_messages_key="chat_history"
)

In [32]:
agent_with_chat_history.invoke(
    {"input": "hi! I'm bob"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "<foo>"}},
)



> Entering new AgentExecutor chain...
Hello Bob! How can I assist you today?

> Finished chain.


{'input': "hi! I'm bob",
 'chat_history': [],
 'output': 'Hello Bob! How can I assist you today?'}

In [34]:
agent_with_chat_history.invoke(
    {"input": "who was the most impactful mau mau freedom fighter?"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "<foo>"}},
)



> Entering new AgentExecutor chain...

Invoking: `maumau_history` with `{'query': 'most impactful Mau Mau freedom fighter'}`


Wunyabari O. Maloba regards the rise of the Mau Mau movement as "without doubt, one of the most important events in recent African history".[105] David Anderson, however, considers Maloba's and similar work to be the product of "swallowing too readily the propaganda of the Mau Mau war", noting the similarity between such analysis and the "simplistic" earlier studies of Mau Mau.[43] This earlier work cast the Mau Mau war in strictly bipolar terms, "as conflicts between anti-colonial nationalists and colonial collaborators".[43] Caroline Elkins' 2005 study, Imperial Reckoning, awarded the 2006 Pulitzer Prize for General Non-Fiction,[106] was also controversial in that she was accused of presenting an equally binary portrayal of the conflict[107] and of drawing questionable conclusions from limited census data, in particular her assertion that the victims of Bri

{'input': 'who was the most impactful mau mau freedom fighter?',
 'chat_history': [HumanMessage(content="hi! I'm bob"),
  AIMessage(content='Hello Bob! How can I assist you today?'),
  HumanMessage(content='who was the most impactful freedom fighter?'),
  AIMessage(content='There have been many impactful freedom fighters throughout history who have fought for the rights and liberation of their people. Some notable freedom fighters include Mahatma Gandhi, Nelson Mandela, Martin Luther King Jr., and Rosa Parks. Each of these individuals made significant contributions to the fight for freedom, justice, and equality in their respective countries. Their actions and leadership have inspired generations of people around the world to stand up against oppression and injustice.')],
 'output': "The Mau Mau movement in Kenya was a significant and impactful struggle for independence from colonial rule. While there were many individuals involved in the movement, some notable figures include Field Ma